In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import threading

# Define the LeNet model
def create_model():
    return models.Sequential([
        layers.Input(shape=(32, 32, 1)),  # Input layer
        layers.Conv2D(6, kernel_size=(5, 5), activation='relu'),  # Conv layer
        layers.AveragePooling2D(pool_size=(2, 2)),               # Pooling layer
        layers.Conv2D(16, kernel_size=(5, 5), activation='relu'), # Conv layer
        layers.AveragePooling2D(pool_size=(2, 2)),               # Pooling layer
        layers.Flatten(),                                        # Flatten
        layers.Dense(120, activation='relu'),                   # Dense layer
        layers.Dense(84, activation='relu'),                    # Dense layer
        layers.Dense(10, activation='softmax')                  # Output layer
    ])

# Dataset preparation
def create_dataset():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    x_train = tf.image.resize(x_train, (32, 32))
    x_test = tf.image.resize(x_test, (32, 32))
    x_train = tf.image.rgb_to_grayscale(x_train) / 255.0
    x_test = tf.image.rgb_to_grayscale(x_test) / 255.0
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(5000).batch(64)
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(64)
    return train_dataset, test_dataset

# Worker class
class Worker:
    def __init__(self, worker_id, model, dataset, parameter_server):
        self.worker_id = worker_id
        self.model = model
        self.dataset = dataset
        self.parameter_server = parameter_server
        self.optimizer = tf.keras.optimizers.Adam()

    def compute_and_send_gradients(self, features, labels):
        with tf.GradientTape() as tape:
            predictions = self.model(features, training=True)
            loss = tf.keras.losses.sparse_categorical_crossentropy(labels, predictions)
            loss = tf.reduce_mean(loss)
        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.parameter_server.receive_gradients(self.worker_id, gradients)
        return loss

    def run(self):
        for step, (features, labels) in enumerate(self.dataset):
            features = tf.convert_to_tensor(features)
            labels = tf.convert_to_tensor(labels)
            loss = self.compute_and_send_gradients(features, labels)
            print(f"Worker {self.worker_id} - Step {step}, Loss: {loss.numpy()}")

# Parameter server
class ParameterServer:
    def __init__(self, model):
        self.model = model
        self.gradient_accumulator = [tf.zeros_like(var) for var in model.trainable_variables]
        self.lock = threading.Lock()

    def receive_gradients(self, worker_id, gradients):
        with self.lock:
            for i, grad in enumerate(gradients):
                self.gradient_accumulator[i] += grad
            print(f"Parameter Server - Received gradients from Worker {worker_id}")

    def apply_gradients(self):
        optimizer = tf.keras.optimizers.Adam()
        with self.lock:
            optimizer.apply_gradients(zip(self.gradient_accumulator, self.model.trainable_variables))
            self.gradient_accumulator = [tf.zeros_like(var) for var in self.model.trainable_variables]
            print("Parameter Server - Gradients applied and parameters updated")

# Parallel training setup
train_dataset, test_dataset = create_dataset()
global_model = create_model()
parameter_server = ParameterServer(global_model)

# Split dataset for workers
worker_datasets = [train_dataset.shard(num_shards=3, index=i) for i in range(3)]
workers = [Worker(worker_id=i, model=create_model(), dataset=worker_datasets[i], parameter_server=parameter_server) for i in range(3)]

# Run workers in parallel
threads = []
for worker in workers:
    thread = threading.Thread(target=worker.run)
    threads.append(thread)
    thread.start()

# Wait for all workers to finish
for thread in threads:
    thread.join()

# Apply accumulated gradients on the parameter server
parameter_server.apply_gradients()

# Evaluate the global model
global_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
test_loss, test_acc = global_model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_acc}")


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Use MirroredStrategy for distributed training simulation
strategy = tf.distribute.MirroredStrategy()

# Define the LeNet model
with strategy.scope():
    model = models.Sequential([
        layers.Input(shape=(32, 32, 1)),  # Input layer
        layers.Conv2D(6, kernel_size=(5, 5), activation='relu'),  # Conv layer
        layers.AveragePooling2D(pool_size=(2, 2)),               # Pooling layer
        layers.Conv2D(16, kernel_size=(5, 5), activation='relu'), # Conv layer
        layers.AveragePooling2D(pool_size=(2, 2)),               # Pooling layer
        layers.Flatten(),                                        # Flatten
        layers.Dense(120, activation='relu'),                   # Dense layer
        layers.Dense(84, activation='relu'),                    # Dense layer
        layers.Dense(10, activation='softmax')                  # Output layer
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Dataset preparation
def create_dataset():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    x_train = tf.image.resize(x_train, (32, 32))
    x_test = tf.image.resize(x_test, (32, 32))
    x_train = tf.image.rgb_to_grayscale(x_train) / 255.0
    x_test = tf.image.rgb_to_grayscale(x_test) / 255.0
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(5000).batch(64)
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(64)
    return train_dataset, test_dataset

# Prepare datasets
train_dataset, test_dataset = create_dataset()

# Distribute the dataset
train_dist_dataset = strategy.experimental_distribute_dataset(train_dataset)
test_dist_dataset = strategy.experimental_distribute_dataset(test_dataset)

# Callback to observe gradients and weights
class GradientCallback(tf.keras.callbacks.Callback):
    def on_train_batch_end(self, batch, logs=None):
        print(f"\n--- Batch {batch} ---")
        for i, weight in enumerate(self.model.trainable_weights):
            print(f"Layer {i} weight shape: {weight.shape}")
            print(f"Sample weight values: {weight.numpy().flatten()[:5]}")  # Display first 5 weights
        print("------------------")

# Function to print data distribution across replicas
@tf.function
def print_batch_distribution(features, labels):
    replica_context = tf.distribute.get_replica_context()
    tf.print(f"Replica {replica_context.replica_id_in_sync_group}: "
             f"Feature shape: {tf.shape(features)}, Label shape: {tf.shape(labels)}")

# Observe how dataset is distributed
iterator = iter(train_dist_dataset)
features, labels = next(iterator)
strategy.run(print_batch_distribution, args=(features, labels))

# Train the model and observe updates
model.fit(train_dist_dataset, epochs=1, callbacks=[GradientCallback()])

# Evaluate the model
loss, accuracy = model.evaluate(test_dist_dataset)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")
